Start with

- Formations (under center, shotgun, pistol)
- Personnel (% 11, % mult TEs, % no TEs, % mult RBs, % no RBs, % extra OL)

- % Pass
- % Pass neutral downs
- QB Scrambles

- ADOT
- % Screens
- % Long
- % passes from play-action
- % passes from under center vs shotgun vs pistol
- number receivers in top 80% targets

- % runs middle, guard/tackle, edge
- % rushes from under center vs. shotgun vs. pistol
- number rushers to account for 20% rushes

In [19]:
''' Imports '''

import pandas as pd
import polars as pl
import numpy as np
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import percentileofscore
from scipy.stats.mstats import trimmed_var

from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler

from prep_data import load_pbp_participation_data, load_stats_team_tendencies_offense, load_stats_team_tendencies_defense

In [2]:
offense_tendencies = load_stats_team_tendencies_offense()

print(offense_tendencies.head().to_string())

                Games  Drives  Plays  Neutral_Down_Plays  Pass_Plays  Neutral_Down_Pass  Pass_Attempts  QBScrambles     IAY  IAY_ToSticks  TotalTimeToThrow  Pass_BehindLOS  Pass_Deep  Sacks  Rush_Plays  Rush_Attempts  Rush_Inside  Rush_Outside  Plays_11_Personnel  Plays_Heavy_Personnel  Plays_Mult_RBs  Plays_Zero_RBs  Plays_Mult_TEs  Plays_Zero_TEs  Plays_Extra_OL  Plays / Game  Drives / Game    % Pass  % Pass Neutral Downs  Scrambles / Game      ADOT  ADOT to Sticks  Avg Time to Throw  % Passes Behind LOS  % Passes Deep  % Rush Inside  % Rush Outside  % Plays 11 Personnel  % Plays Heavy Personnel  % Plays Mult RBs  % Plays Zero RBs  % Plays Mult TEs  % Plays Zero TEs  % Plays Extra OL  Shotgun Plays  Under Center Plays  Shotgun Neutral_Down_Plays  Under Center Neutral_Down_Plays  Shotgun % Pass  Under Center % Pass  % Under Center  % Shotgun  % Under Center Neutral Downs  % Shotgun Neutral Downs  MaxTargets  MaxTargetShare  N_Receivers_FivePctTargetShare  MaxRushAttempts  MaxRushAttem

In [5]:
''' Features '''
# '% Pass Neutral Downs', '% Under Center Neutral Downs', '% Shotgun Neutral Downs',

OFFENSE_FEATURES = [
    'Plays / Game', 'Drives / Game', 
    '% Pass',  'Scrambles / Game',
    '% Plays 11 Personnel', '% Plays Mult RBs', '% Plays Zero RBs', '% Plays Mult TEs', '% Plays Zero TEs', '% Plays Extra OL',
    '% Under Center', '% Shotgun', 'Shotgun % Pass', 'Under Center % Pass',
    'ADOT', 'ADOT to Sticks', 'Avg Time to Throw', '% Passes Behind LOS', '% Passes Deep', 'MaxTargetShare',
    '% Rush Inside', '% Rush Outside', 'MaxRushAttemptsShare',
]

VIZ_FEATURES = ['Plays / Game', '% Pass', 'Scrambles / Game', 
                '% Plays 11 Personnel', '% Plays Heavy Personnel',
                '% Under Center', 'ADOT', 'Avg Time to Throw', 'MaxTargetShare', 
                '% Rush Outside', 'MaxRushAttemptsShare']

# VIZ_FEATURES = ['Plays / Game', '% Pass', 'Scrambles / Game', 
#                 '% Plays Plays_11_Personnel', '% Plays Plays_Mult_RBs', '% Plays Plays_Mult_TEs',
#                 '% Under Center', 'Shotgun % Pass', 'Under Center % Pass', 
#                 'ADOT', 'Avg Time to Throw', '% Passes Behind LOS', '% Passes Deep', 'MaxTargetShare', 
#                 '% Rush Outside', 'MaxRushAttemptsShare']

# Visualize

In [6]:
''' Correlation Matrix '''

corr_matrix = offense_tendencies[OFFENSE_FEATURES].corr()

fig = px.imshow(
    corr_matrix,
    color_continuous_scale=px.colors.diverging.PRGn,
    aspect="auto"
)
fig.update_xaxes(side="top")
fig.update_coloraxes(
    cmid=0,
    showscale=False,
)
fig.update_layout(
    title='Feature Correlations',
    margin=dict(r=25, b=25)
)
fig.show()

In [7]:
''' Variance '''

# Calculate variance, get 10 largest features
top_ten_variance = offense_tendencies[OFFENSE_FEATURES].var().sort_values().tail(10)

# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_variance,
    y=top_ten_variance.index,
    title="Offense Tendencies: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Features")
fig.show()

# Calculate trimmed variance
top_ten_trim_variance = offense_tendencies[OFFENSE_FEATURES].apply(trimmed_var).sort_values().tail(10)

# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_variance,
    y=top_ten_trim_variance.index,
    title="Offense Tendencies: High Variance Features (Trimmed)"
)
fig.update_layout(xaxis_title="Trimmed Variance", yaxis_title="Features")
fig.show()


In [8]:
fig = px.histogram(
    data_frame=offense_tendencies,
    x='Plays / Game',
    title='Plays / Game'
)
fig.show()

# Model Preprocessing

In [9]:
''' Transform and Scale '''

# ## Log transform data
# transformed_data = pd.DataFrame(np.log(offense_tendencies[OFFENSE_FEATURES]), columns=OFFENSE_FEATURES).replace(math.inf, 0).replace(-(math.inf), 0)

## Scale data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(offense_tendencies[OFFENSE_FEATURES])

# Put `scaled_data` into DataFrame
scaled_data_df = pd.DataFrame(scaled_data, columns=OFFENSE_FEATURES)

print("scaled DF type:", type(scaled_data_df))
print("scaled DF shape:", scaled_data_df.shape)
scaled_data_df.head()

scaled DF type: <class 'pandas.core.frame.DataFrame'>
scaled DF shape: (224, 23)


,Plays / Game,Drives / Game,% Pass,Scrambles / Game,% Plays 11 Personnel,% Plays Mult RBs,% Plays Zero RBs,% Plays Mult TEs,% Plays Zero TEs,% Plays Extra OL,...,Under Center % Pass,ADOT,ADOT to Sticks,Avg Time to Throw,% Passes Behind LOS,% Passes Deep,MaxTargetShare,% Rush Inside,% Rush Outside,MaxRushAttemptsShare
0,-2.384848,1.301345,0.375356,-0.672259,0.643966,0.069819,-0.503830,-0.429453,-0.493944,-0.714952,...,0.450339,0.161504,-0.218168,-0.505466,0.314451,0.184435,0.121146,-2.280538,-1.670193,1.975035
1,-0.229150,0.428337,0.640419,0.384403,-2.178527,-0.427404,0.401957,-0.064989,8.333983,-0.625306,...,0.157428,-0.390357,-0.437113,-0.331755,1.341118,0.607310,-0.611336,-2.786693,2.174893,-1.348458
2,1.473851,0.646589,-0.074467,1.507106,-1.415593,-0.486232,-0.474154,0.320762,4.733059,0.526516,...,0.791641,-0.045698,-0.151606,-0.407897,0.508219,0.358306,1.491913,-2.413942,0.447978,0.366748
3,0.939998,-0.483186,-0.273242,0.497062,-0.474332,-0.390634,-0.158507,-0.196691,3.326573,-0.444512,...,-0.407156,-0.541253,-0.556292,-0.512009,1.745810,0.801320,-1.224896,-2.049898,-0.015892,-0.529315
4,1.548666,0.235761,1.003423,0.870001,0.172832,-0.438580,-0.547651,-0.027959,1.087429,-0.673022,...,-1.455882,-1.117789,-1.238260,-1.351201,1.138543,-0.215576,-1.520563,-1.217615,-0.685495,-0.374213


In [10]:
''' Variance of Scaled / Transformed Data '''

# Calculate variance, get 10 largest features
top_ten_variance = scaled_data_df.var().sort_values().tail(10)

# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_variance,
    y=top_ten_variance.index,
    title="Scaled Data: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Features")
fig.show()

# PCA

In [11]:
''' PCA '''

# Instantiate transformer
pca = PCA(random_state=42)

# Transform data with pa
pca_component_data = pca.fit_transform(scaled_data_df)

print('Total variance:', scaled_data_df.var().sum())
print(f'Singular values:\n', pca.singular_values_)
print(f'Explained variance:\n', pca.explained_variance_.round(5))
print(f'Ratio:\n', pca.explained_variance_ratio_.round(3))
print(pca.feature_names_in_)

# Create horizontal bar chart of explained variance
fig = px.line(
    x=[i + 1 for i in range(len(pca.explained_variance_ratio_))],
    y=pca.explained_variance_ratio_.cumsum(),
    title="Explained variance"
)
fig.update_layout(xaxis_title="Principal Component", yaxis_title="Cumulative Explained Variance (%)")
fig.show()

Total variance: 23.103139013452914
Singular values:
 [2.79343569e+01 2.60516581e+01 2.37569696e+01 1.99175021e+01
 1.88453271e+01 1.78005208e+01 1.65466538e+01 1.61870289e+01
 1.55076544e+01 1.42082171e+01 1.38616082e+01 1.37576501e+01
 1.20972884e+01 1.13114193e+01 1.11730160e+01 1.03872474e+01
 9.52316446e+00 8.06068031e+00 4.84452081e+00 2.70897552e+00
 2.06837861e+00 1.58516077e+00 2.11103632e-15]
Explained variance:
 [3.49923 3.04345 2.53091 1.77895 1.59258 1.42089 1.22777 1.17498 1.07842
 0.90526 0.86163 0.84876 0.65625 0.57376 0.5598  0.48383 0.40668 0.29137
 0.10524 0.03291 0.01918 0.01127 0.     ]
Ratio:
 [0.151 0.132 0.11  0.077 0.069 0.062 0.053 0.051 0.047 0.039 0.037 0.037
 0.028 0.025 0.024 0.021 0.018 0.013 0.005 0.001 0.001 0.    0.   ]
['Plays / Game' 'Drives / Game' '% Pass' 'Scrambles / Game'
 '% Plays 11 Personnel' '% Plays Mult RBs' '% Plays Zero RBs'
 '% Plays Mult TEs' '% Plays Zero TEs' '% Plays Extra OL' '% Under Center'
 '% Shotgun' 'Shotgun % Pass' 'Under Cen

In [12]:
''' PCA - final '''

# Set number of PCA components to use after initial try
PCA_N_COMPONENTS = 3
COMPONENT_COLS = [f'Component {n}' for n in range(1, PCA_N_COMPONENTS + 1)]

# Instantiate transformer
pca_final = PCA(n_components=PCA_N_COMPONENTS, random_state=42)

# Transform sku profiles
pca_component_data_final = pca_final.fit_transform(scaled_data_df)

# Evaluate components
total_variance = scaled_data_df.var().sum()
expl_variance = pca_final.explained_variance_.sum()

print(f'Data set variance: {total_variance:,.3f}')
print(f'PCA explained variance: {expl_variance:,.3f} ({round((expl_variance / total_variance) * 100, 2)}%)')

pcs = pd.DataFrame(pca_final.components_, columns=OFFENSE_FEATURES)

# Create bar charts of contribution
for n in range(2):      #PCA_N_COMPONENTS):
    pc = pcs.transpose()[n].sort_values(ascending=False)

    fig = px.bar(
        x=pc,
        y=pc.index,
        title=f"PC{n+1}: Greatest contributors"
    )
    fig.update_layout(
        xaxis_title="Correlation", 
        yaxis_title="Features", 
        yaxis={'dtick': 1, 'categoryorder':'total ascending'},
    )
    fig.show()

    comp_expl_variance = pca_final.explained_variance_[n]
    print(f'Explained variance: {comp_expl_variance:,} ({round((comp_expl_variance / total_variance) * 100, 2)}%)')
                                                                                                                                                                                                              
# Make df of PCA scores
pca_component_df = pd.DataFrame(data=pca_component_data_final, columns=[f'Component {i}' for i in range(1, PCA_N_COMPONENTS + 1)])
print(pca_component_df.shape)
print(pca_component_df.head().to_string())

# Add PCA scores to original dataframe
offense_tendencies = offense_tendencies.drop(columns=list(filter(lambda x: x.startswith("Component"), offense_tendencies.columns)))
offense_tendencies = offense_tendencies.reset_index().merge(pca_component_df, left_index=True, right_index=True, how='left').set_index(['posteam', 'season'])


print(f'PCA values')
print(offense_tendencies.head().to_string())

Data set variance: 23.103
PCA explained variance: 9.074 (39.27%)


Explained variance: 3.499230030926453 (15.15%)


Explained variance: 3.043447946211489 (13.17%)
(224, 3)
   Component 1  Component 2  Component 3
0     1.820424     0.704133     1.299442
1    -2.711398    -3.010687    -0.729077
2    -3.285122    -2.258667     0.039813
3    -2.641523    -3.322270    -0.066368
4    -1.342982    -3.941394     1.126703
PCA values
                Games  Drives  Plays  Neutral_Down_Plays  Pass_Plays  Neutral_Down_Pass  Pass_Attempts  QBScrambles     IAY  IAY_ToSticks  TotalTimeToThrow  Pass_BehindLOS  Pass_Deep  Sacks  Rush_Plays  Rush_Attempts  Rush_Inside  Rush_Outside  Plays_11_Personnel  Plays_Heavy_Personnel  Plays_Mult_RBs  Plays_Zero_RBs  Plays_Mult_TEs  Plays_Zero_TEs  Plays_Extra_OL  Plays / Game  Drives / Game    % Pass  % Pass Neutral Downs  Scrambles / Game      ADOT  ADOT to Sticks  Avg Time to Throw  % Passes Behind LOS  % Passes Deep  % Rush Inside  % Rush Outside  % Plays 11 Personnel  % Plays Heavy Personnel  % Plays Mult RBs  % Plays Zero RBs  % Plays Mult TEs  % Plays Zero TEs  % Plays E

In [13]:
''' Visualize PCA - Correlation Chart '''

# COMPONENT_NAMES = ['Small Cartons', 'High Volume', 'High Inventory - Light', 'High Inventory - Heavy']
COMPONENT_NAMES = COMPONENT_COLS

# Create "correlation matrix" - correlation with each of the components to each of the features
corr_matrix = pd.DataFrame.from_records(pca_final.components_, index=COMPONENT_NAMES, columns=OFFENSE_FEATURES).transpose()

# Visualize
fig = px.imshow(
    corr_matrix,
    color_continuous_scale=px.colors.diverging.PRGn,
    aspect="auto"
)
fig.update_xaxes(side="top")
fig.update_coloraxes(
    showscale=False,
    cmid=0,
)
fig.update_layout(
    title='Offense Tendency Factors',
    margin=dict(r=25, b=25, t=75),
    height=700,
    width=900

)
fig.show()

In [14]:
''' Visualize PCA - Items '''

# Percentile each component
for n in range(1, PCA_N_COMPONENTS + 1):
    offense_tendencies[f'Component {n} Percentile'] = offense_tendencies[f'Component {n}'].rank(pct=True)

COMPONENT_PERCENTILES = [f'Component {n} Percentile' for n in range(1, PCA_N_COMPONENTS + 1)]

def visualize_team_pca(team: str, season: int):

    ## Data ##
    # Get slice from offensive tendencies
    team_sl = offense_tendencies.loc[(offense_tendencies.index.get_level_values('posteam') == team) &
                                     (offense_tendencies.index.get_level_values('season') == season), :]
    
    # PCA Component %iles
    team_component_pct_ranks = team_sl[COMPONENT_PERCENTILES].values.tolist()[0]

    # Feature values
    team_feature_vals = team_sl[OFFENSE_FEATURES].values.tolist()[0]
    
    # Feature value percentiles
    vals_fmt = []
    pct_scores = []
    for i in range(len(OFFENSE_FEATURES)):
        feature = OFFENSE_FEATURES[i]
        val = team_feature_vals[i]
        pct_score = percentileofscore(offense_tendencies[feature].tolist(), val, kind='weak') / 100
        
        val_fmt = f'{val:.1%}' if feature[0] == '%' else f'{val:.2f}'
        vals_fmt.append(val_fmt)
        pct_scores.append(f'{pct_score:.1%}')

    ## Figure ##

    fig = make_subplots(
        rows=1, cols=2, 
        column_widths=[4,3],
        horizontal_spacing=0.1,
        specs=[[{"type": "polar"}, {"type": "domain"}]]
    )

    fig.add_trace(
        go.Scatterpolar(
            r=team_component_pct_ranks,
            theta=COMPONENT_NAMES,
            opacity=0.7,
            fill='toself'
        ),
        row=1, col=1
    )
    fig.update_layout(
        title_text=f"Team: {season} {team}",
        polar=dict(radialaxis_range=(0,1)),
        margin=dict(b=50, r=50, l=75, t=75)
    )

    fig.add_trace(
        go.Table(
            columnwidth=[2,1,1],
            header={
                "values": ['Component', 'Value', 'Percentile'],
            },
            cells={
                "values": [OFFENSE_FEATURES, vals_fmt, pct_scores]
            }
        ),
        row=1, col=2
    )

    fig.show()


visualize_team_pca('DET', 2024)

# # Visualize top teams from each component
# for n in range(1, PCA_N_COMPONENTS + 1):
#     component = f'Component {n} Percentile'

#     top_teams = offense_tendencies.sort_values(by=component, ascending=False).head(10)
    
#     top_team = top_teams.index[0]
#     visualize_team_pca(top_team[0], top_team[1])

#     print(top_teams.to_string())

In [15]:
fig = px.histogram(
    data_frame=offense_tendencies,
    x='MaxTargetShare'
)
fig.show()

In [18]:
''' Visualize PCA Components - Scatter '''

# sl = offense_tendencies.sample(frac=0.25, random_state=42)

fig = px.scatter(
    data_frame=offense_tendencies,
    x='Component 1',
    y='Component 2',
    title='PCA Components - Top 2'
)
fig.show()

fig = px.scatter_3d(
    data_frame=offense_tendencies,
    x='Component 1',
    y='Component 2',
    z='Component 3',
    title='PCA Components - Top 3'
)
fig.show()

# t-SNE

In [47]:
''' t-SNE '''

TSNE_N_COMPONENTS = 3
TSNE_N_COMPONENT_NAMES = [f'TSNE Component {n+1}' for n in range(TSNE_N_COMPONENTS)]

perplexities = [i for i in range(5, 100, 5)]
divergences = []

for p in perplexities:
    # Model
    tsne_model = TSNE(n_components=TSNE_N_COMPONENTS, perplexity=p, random_state=42)
    
    # Fit
    y = tsne_model.fit_transform(scaled_data_df)

    # Divergence
    divergences.append(tsne_model.kl_divergence_)

# Graph divergence
fig = px.line(
    x=perplexities, 
    y=divergences,
    markers=True
)
fig.update_layout(xaxis_title="Perplexity Values", yaxis_title="Divergence")
fig.update_traces(line_color="red", line_width=1)
fig.show()

In [48]:
''' t-SNE Final '''

PERPLEXITY = 85

# Model
tsne_model = TSNE(n_components=TSNE_N_COMPONENTS, perplexity=PERPLEXITY, random_state=42)

# Fit
y = tsne_model.fit_transform(scaled_data_df)
print(f'Divergence:', tsne_model.kl_divergence_)

# Results df
tsne_df = pd.DataFrame(y, columns=TSNE_N_COMPONENT_NAMES)

print(tsne_df.shape)
print(tsne_df.head().to_string())

Divergence: 0.6142880916595459
(224, 3)
   TSNE Component 1  TSNE Component 2  TSNE Component 3
0        -18.391388         -5.919055        -34.658665
1         23.835779          8.242852         56.953773
2         21.069798         -3.894374         51.772396
3         32.970856         -0.120803         46.860619
4         38.794247         -7.056817         35.840225


In [ ]:
fig = px.scatter_3d(
    data_frame=tsne_df,
    x='TSNE Component 1',
    y='TSNE Component 2',
    z='TSNE Component 3',
)
fig.show()

In [50]:
''' Kmeans Clustering '''

kmeans_input = tsne_df

# Try kmeans clustering with up to 20 clusters, keep track of inertia (basically cluster variance)
n_clusters = range(2,10)
inertia_values = []
silhouette_scores = []

for i in n_clusters:
    # Model
    kmeans = KMeans(n_clusters=i, n_init='auto', init='k-means++', random_state=42)

    # Fit
    kmeans.fit(kmeans_input)

    # Score
    ss = silhouette_score(kmeans_input, kmeans.labels_)   #, sample_size=int(len(pca_component_df) * 0.25))

    inertia_values.append(kmeans.inertia_)
    silhouette_scores.append(ss)

# Create scatter of inertia
fig = px.line(
    x=[i + 1 for i in range(len(inertia_values))],
    y=inertia_values,
    title="Kmeans - Inertia by Number of Clusters"
)
fig.update_layout(xaxis_title="Num Clusters", yaxis_title="Inertia")
fig.show()

# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters,
    y=silhouette_scores,
    title="K-Means Model: Silhouette Score vs Number of Clusters"
)
fig.update_layout(xaxis_title="Num Clusters", yaxis_title="Silhouette Score")
fig.show()

In [53]:
N_CLUSTERS = 4

# Once optimal num clusters is found, create the final cluster model
kmeans_final = KMeans(n_clusters=N_CLUSTERS, n_init='auto', init='k-means++', random_state=42)
kmeans_final.fit(kmeans_input)

# Find distances to centroids
labels = kmeans_final.labels_
distances_array = kmeans_final.transform(kmeans_input)

tsne_df['Cluster KMEANS'] = labels + 1
tsne_df['Cluster KMEANS'] = tsne_df['Cluster KMEANS'].astype(str)

print(tsne_df.head())

   TSNE Component 1  TSNE Component 2  TSNE Component 3 Cluster KMEANS
0        -18.391388         -5.919055        -34.658665              3
1         23.835779          8.242852         56.953773              4
2         21.069798         -3.894374         51.772396              4
3         32.970856         -0.120803         46.860619              2
4         38.794247         -7.056817         35.840225              2


In [54]:
fig = px.scatter_3d(
    data_frame=tsne_df,
    x='TSNE Component 1',
    y='TSNE Component 2',
    z='TSNE Component 3',
    color='Cluster KMEANS',
)
fig.show()